In [1]:
%run "C:\Users\james\Documents\MLB\Code\U1. Imports-WIP.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U2. Utilities.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"

### Games

In [2]:
start_date = yesterdaysdate
end_date = yesterdaysdate

Read in games

In [3]:
game_df = read_and_save_games(team_map, generate=True)

In [4]:
game_df = game_df[(game_df['date'] >= start_date) & (game_df['date'] <= end_date)].reset_index(drop=True)

### A05. Rosters

In [5]:
%run "C:\Users\james\Documents\MLB\Code\A05. Rosters.ipynb"

##### 1. Batting Orders

In [6]:
# Run in parallel
empty_list = Parallel(n_jobs=-1, verbose=0)(delayed(orders)(team_map, game_df, row) for row in range(len(game_df)))

##### 2. Rosters

In [7]:
# Run in parallel
empty_list = Parallel(n_jobs=-1, verbose=0)(delayed(rosters)(team_map, game_df, row) for row in range(len(game_df)))

### W02. Park Factors

In [8]:
%run "C:\Users\james\Documents\MLB\Code\W02. Park Factors.ipynb"

b1
b2
b3
hr
bb
hbp
so
fo
go
lo
po
b1
b2
b3
hr
bb
hbp
so
fo
go
lo
po


### A06. Weather

In [9]:
%run "C:\Users\james\Documents\MLB\Code\A06. Weather-WIP.ipynb"

In [10]:
# Read in datasets
multiplier_dataset = pd.read_csv(os.path.join(baseball_path, "Multiplier Dataset.csv"))
period_avg_df = pd.read_csv(os.path.join(baseball_path, "Period Averages.csv"))

# Calculate park x weather factor
daily_weather_df = park_and_weather_factors(game_df, multiplier_dataset, period_avg_df, swish_df=None, date=yesterdaysdate, overwrite_year=None, historic=True)

# To csv
daily_weather_df.to_csv(os.path.join(baseball_path, "A06. Weather", "4. Park and Weather Factors", f'Park and Weather Factors {yesterdaysdate}.csv'), index=False)

20240612


### A08. Odds Sportsbook Review

In [11]:
%run "C:\Users\james\Documents\MLB\Code\A08. Odds Sportsbook Review.ipynb"

In [12]:
# Scrape odds from Sportsbook Review
odds_df = create_odds_df(yesterdaysdate_dash)

# Save 
odds_df.to_csv(os.path.join(baseball_path, "A08. Odds Sportsbook Review", f"Odds {yesterdaysdate}.csv"), index=False)

### A09. Contest Guides

In [13]:
%run "C:\Users\james\Documents\MLB\Code\A09. Contest Guides.ipynb"

In [14]:
# Read in yesterdays subset
yesterdays_subset_df = pd.read_csv(os.path.join(baseball_path, 'A01. DraftKings', '7. Subsets', f'Subset {yesterdaysdate}.csv'))

In [15]:
# Loop over contestKeys
yesterdays_subset_df = yesterdays_subset_df.reset_index(drop=True)

for contestKey in yesterdays_subset_df['contestKey'].reset_index(drop=True):
    print(contestKey)
    try:
        guide = contest_guide(game_df, subset_df=yesterdays_subset_df, contestKey=contestKey)
        if not guide.empty:
            guide.to_csv(os.path.join(baseball_path, "A09. Contest Guides", f"Contest Guide {contestKey}.csv"), index=False)
        else:
            print(f"Contest Guide {contestKey} is empty.")
    except FileNotFoundError as e:
        print(f"Draftables {contestKey}.csv not found.")

162564050
162564049
162563936
162577090
162576660
162563925
Draftables 162563925.csv not found.
162563924
Draftables 162563924.csv not found.
162563923
Draftables 162563923.csv not found.
162563922
Draftables 162563922.csv not found.
162563921
Draftables 162563921.csv not found.
162564096
162564094
162564098
162564089
162563020
162563872
Draftables 162563872.csv not found.
162563871
Draftables 162563871.csv not found.
162563870
Draftables 162563870.csv not found.
162563868
Draftables 162563868.csv not found.
162563867
Draftables 162563867.csv not found.
162564079
162564082
162564086
162564074
162564078
162563908
Draftables 162563908.csv not found.
162563907
Draftables 162563907.csv not found.
162563906
Draftables 162563906.csv not found.
162563905
Draftables 162563905.csv not found.
162563904
Draftables 162563904.csv not found.
162564073
162564068
162562749
162563636
162564069
162564063
162564059
162563460
162562699
162564061
162563821
Draftables 162563821.csv not found.
162563820
Draf

### A10. Player Results

In [16]:
%run "C:\Users\james\Documents\MLB\Code\A10. Player Results.ipynb"

In [17]:
%%time
for i in range(len(game_df)):
    run_result_dfs(i)

745896
745978
745328
745405
747028
746462
745079
746701
746948
745811
745171
746294
747193
745246
746137
CPU times: total: 7.58 s
Wall time: 15 s


### B01. Matchups

In [18]:
%run "C:\Users\james\Documents\MLB\Code\A02. MLB API-WIP.ipynb"
%run "C:\Users\james\Documents\MLB\Code\A03. Steamer.ipynb"
%run "C:\Users\james\Documents\MLB\Code\B01. Matchups.ipynb"

In [19]:
%%time
# Read in dataset 
complete_dataset = create_pa_inputs(park_factors, team_map, 2015, 2024, 50, 300, True)
# Subset
complete_dataset = complete_dataset.query('date > 20220301')

# Read in Steamer hitters
steamer_hitters_df = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_hitters_weekly_log.csv"), encoding='iso-8859-1')
steamer_hitters_df_current = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_hitters.csv"), encoding='iso-8859-1')
steamer_hitters_df = pd.concat([steamer_hitters_df, steamer_hitters_df_current], axis=0)
steamer_hitters_df['proj_year'].fillna(2024, inplace=True)
steamer_hitters_df['proj_date'].fillna(todaysdate_dash, inplace=True)
steamer_hitters_df = clean_steamer_hitters(steamer_hitters_df)

# Read in Steamer pitchers
steamer_pitchers_df = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_pitchers_weekly_log.csv"), encoding='iso-8859-1')
steamer_pitchers_df_current = pd.read_csv(os.path.join(baseball_path, "A03. Steamer", "steamer_pitchers.csv"), encoding='iso-8859-1')
steamer_pitchers_df = pd.concat([steamer_pitchers_df, steamer_pitchers_df_current], axis=0)
steamer_pitchers_df['proj_year'].fillna(2024, inplace=True)
steamer_pitchers_df['proj_date'].fillna(todaysdate_dash, inplace=True)
steamer_pitchers_df = clean_steamer_pitchers(steamer_pitchers_df)

CPU times: total: 6min 17s
Wall time: 8min 9s


In [20]:
# Write to CSV (we'll read these later in B.)
complete_dataset.to_csv(os.path.join(baseball_path, "Stat Dataset.csv"), index=False)
steamer_hitters_df.to_csv(os.path.join(baseball_path, "Steamer Hitters.csv"), index=False)
steamer_pitchers_df.to_csv(os.path.join(baseball_path, "Steamer Pitchers.csv"), index=False)

In [22]:
%%time
print(len(game_df))
empty_list = Parallel(n_jobs=-1, verbose=True)(delayed(create_matchup_files)(game_df, row, complete_dataset, steamer_hitters_df, steamer_pitchers_df, team_map) for row in range(len(game_df)))

15


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.3min finished


CPU times: total: 51.5 s
Wall time: 1min 18s
